# Sentiment Analysis NLP Project
## Interactive Colab Notebook

This notebook demonstrates sentiment analysis using our custom analyzer.

## 1. Setup and Installation

In [ ]:
# Install required packages
!pip install nltk textblob -q

# Download NLTK data
import nltk
nltk.download('punkt', quiet=True)
print("✓ Setup complete!")

## 2. Clone Repository (Optional)

If you want to use the code from GitHub:

In [ ]:
# Uncomment to clone from GitHub
# !git clone https://github.com/yourusername/nlp-project.git
# import sys
# sys.path.append('/content/nlp-project/src')

## 3. Import Sentiment Analyzer

In [ ]:
# For this demo, we'll define the analyzer inline
import nltk
from textblob import TextBlob
from typing import Dict, List
import re

class SentimentAnalyzer:
    def __init__(self):
        """Initialize the sentiment analyzer"""
        try:
            nltk.data.find('tokenizers/punkt')
        except LookupError:
            nltk.download('punkt', quiet=True)
    
    def preprocess_text(self, text: str) -> str:
        """Clean and preprocess text"""
        text = text.lower()
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
        text = re.sub(r'[^\w\s.,!?]', '', text)
        return text.strip()
    
    def analyze_sentiment(self, text: str) -> Dict[str, float]:
        """Analyze sentiment of text"""
        cleaned_text = self.preprocess_text(text)
        blob = TextBlob(cleaned_text)
        
        return {
            'polarity': blob.sentiment.polarity,
            'subjectivity': blob.sentiment.subjectivity,
            'label': self._get_sentiment_label(blob.sentiment.polarity)
        }
    
    def _get_sentiment_label(self, polarity: float) -> str:
        """Convert polarity score to label"""
        if polarity > 0.1:
            return 'positive'
        elif polarity < -0.1:
            return 'negative'
        else:
            return 'neutral'
    
    def batch_analyze(self, texts: List[str]) -> List[Dict[str, float]]:
        """Analyze sentiment for multiple texts"""
        return [self.analyze_sentiment(text) for text in texts]

# Initialize analyzer
analyzer = SentimentAnalyzer()
print("✓ Sentiment Analyzer ready!")

## 4. Basic Examples

In [ ]:
# Test with sample texts
sample_texts = [
    "I absolutely love this product! It's amazing!",
    "This is terrible and disappointing.",
    "It's okay, nothing special.",
    "The customer service was excellent and very helpful!",
    "Worst experience ever. Never coming back."
]

print("Sentiment Analysis Results:\n" + "="*60)
for text in sample_texts:
    result = analyzer.analyze_sentiment(text)
    print(f"\n📝 Text: {text}")
    print(f"   Sentiment: {result['label'].upper()}")
    print(f"   Polarity: {result['polarity']:.3f}")
    print(f"   Subjectivity: {result['subjectivity']:.3f}")

## 5. Interactive Analysis

Try your own text!

In [ ]:
# Analyze your own text
user_text = input("Enter text to analyze: ")
result = analyzer.analyze_sentiment(user_text)

print(f"\n{'='*60}")
print(f"Sentiment: {result['label'].upper()}")
print(f"Polarity Score: {result['polarity']:.3f} (range: -1 to 1)")
print(f"Subjectivity Score: {result['subjectivity']:.3f} (range: 0 to 1)")
print(f"{'='*60}")

## 6. Visualization

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Analyze multiple texts and visualize
texts = [
    "Excellent product!",
    "Pretty good overall",
    "It's just okay",
    "Not great",
    "Terrible quality"
]

results = analyzer.batch_analyze(texts)
polarities = [r['polarity'] for r in results]
subjectivities = [r['subjectivity'] for r in results]

# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Polarity chart
colors = ['green' if p > 0.1 else 'red' if p < -0.1 else 'gray' for p in polarities]
ax1.barh(range(len(texts)), polarities, color=colors, alpha=0.7)
ax1.set_yticks(range(len(texts)))
ax1.set_yticklabels([t[:30] + '...' if len(t) > 30 else t for t in texts])
ax1.set_xlabel('Polarity Score')
ax1.set_title('Sentiment Polarity Analysis')
ax1.axvline(x=0, color='black', linestyle='--', linewidth=0.5)
ax1.grid(axis='x', alpha=0.3)

# Scatter plot
ax2.scatter(polarities, subjectivities, c=colors, s=100, alpha=0.6)
ax2.set_xlabel('Polarity')
ax2.set_ylabel('Subjectivity')
ax2.set_title('Polarity vs Subjectivity')
ax2.grid(alpha=0.3)
ax2.axvline(x=0, color='black', linestyle='--', linewidth=0.5)

plt.tight_layout()
plt.show()

print("\n📊 Visualization complete!")

## 7. Batch Processing Example

In [ ]:
# Simulate processing multiple reviews
reviews = [
    "Amazing quality and fast shipping!",
    "Product arrived damaged.",
    "Good value for money.",
    "Not as described, very disappointed.",
    "Exceeded my expectations!",
    "Average product, nothing special.",
    "Great customer support!",
    "Poor quality materials."
]

results = analyzer.batch_analyze(reviews)

# Summary statistics
positive_count = sum(1 for r in results if r['label'] == 'positive')
negative_count = sum(1 for r in results if r['label'] == 'negative')
neutral_count = sum(1 for r in results if r['label'] == 'neutral')

print("\n📊 Batch Analysis Summary:")
print(f"Total reviews: {len(reviews)}")
print(f"Positive: {positive_count} ({positive_count/len(reviews)*100:.1f}%)")
print(f"Negative: {negative_count} ({negative_count/len(reviews)*100:.1f}%)")
print(f"Neutral: {neutral_count} ({neutral_count/len(reviews)*100:.1f}%)")
print(f"\nAverage polarity: {np.mean([r['polarity'] for r in results]):.3f}")

## 8. Export Results

In [ ]:
import pandas as pd

# Create DataFrame with results
df = pd.DataFrame({
    'text': reviews,
    'sentiment': [r['label'] for r in results],
    'polarity': [r['polarity'] for r in results],
    'subjectivity': [r['subjectivity'] for r in results]
})

print(df)

# Save to CSV
df.to_csv('sentiment_results.csv', index=False)
print("\n✓ Results saved to sentiment_results.csv")